# Electronic Spectroscopy with VOTCA and LAMMPS

In this tutorial we will investigate the solvatochromic shift in the spectrum of acetone in water using LAMMPS and VOTCA.

## Before we start

VOTCA has a command line interface, but no python interface (yet!). We don't use jupyter for its python capabilities in this tutorial, but for its ability to show and run commands with comments in Markdown. Some things you need to know

* The exclamation mark (`!`) runs the cell as a Linux/Unix command in the terminal, hence all lines starting with `!` are not python, but shell commands and can also be run in the terminal (or docker container) directly.
* VOTCA requires certain environment variables to be set. This is done with the `VOTCARC.bash` or `VOTCARC.csh` script in the `bin` directory of VOTCA. It needs to be sourced before the jupyter notebook server is started or any VOTCA related commands are executed in the terminal. As an example: suppose VOTCA is installed in `/home/user/Software/votca`, then to set all environment variables and start jupyter run
```bash
source /home/user/Software/votca/bin/VOTCARC.bash
jupyter notebook
```
In the docker container you only need to run
```bash
source VOTCARC.bash
```

### Testing the environment
To check if the environment variables are set correctly, we will execute the `xtp_tools` command with the help flag `-h`. The output should be the VOTCA banner and a list of allowed options. If you get something else make sure you sourced the `VOTCARC` script.

In [2]:
!xtp_tools -h

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: 1754257 (dirty) (compiled May  3 2021, 13:21:18)
votca_csg, version 2022-dev gitid: 1754257 (dirty) (compiled Apr 28 2021, 11:28:58)
votca_tools, version 2022-dev gitid: 1754257 (dirty) (compiled Apr 28 2021, 11:28:24)

Runs excitation/charge transport tools



Allowed options:
  -h [ --help ]                 display this help and exit
  --verbose                     be loud and noisy
  --verbose1                    be very loud and noisy
  -v [ --verbose2 ]             be extremly loud and noisy
  -o [ --options ] arg          calculator options
  -t [ --nthreads ] arg (=1)    number of threads to create

Tools:
  -e [ --execute ] arg        name of the tool to run
  -l [ --list ]               Lists all available tools
  -d [ --description ] arg    Short description of a tool
  -n [ --name ] arg           Name of the job to run



## Creating the topology and parametrizing the model

As a starting point for the QM/MM calculations we need a topology (i.e. the layout of the system). This topology is not just a snap shot of an MD trajectory, eventhough it is a major part of it. Besides the MD trajectory, we need information about

1. The ground state geometry of the molecules,
2. The multipole representation of the molecules and
3. Information about the polarizabililty of the molecules.

This information can be obtained in different ways. For this tutorial we have chosen to use the ORCA dft package, for which VOTCA has an interface.  The necessary files and the ORCA input files that generated them can be found in the `DFT_ORCA` folder. 

### 1. Getting the ground state geometry of a single molecule

To compute the ground state geometry, a DFT optimization calculation needs to be performed. In this tutorial we have done this with ORCA. The input files and optimized geometry can be found in the `DFT_ORCA` folder.

### 2. The multipole representation: converting the ORCA files to VOTCA input files

To get the multipole representation of the molecules we use partial charges computed with the CHELPG scheme of ORCA. We need to convert the ORCA CHELPG output to VOTCA multipole files (.mps). This is done with the following command. 


In [2]:
!xtp_tools -e log2mps -o OPTIONS/log2mps_water.xml

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 09:04:09)
votca_csg, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 08:59:16)
votca_tools, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 08:58:17)

Initializing tool
... log2mps 
... ... DFT_ORCA/acetone/chelpg.log => MP_FILES/acetone_n.mps
Evaluating tool
... log2mps  Using 1 threads
... ... Using package <orca>
... ... Parsing DFT_ORCA/acetone/chelpg.log
... ... Getting the coordinates
... ... Getting charges
... ... Getting charges
... ... 10 QM atoms, total charge Q = 1e-06

The previous command shows the general structure of running a program, which is always the same

```bash
xtp_<executableType> -e <whichCalculationToRun> -o <optionsFile> ...
```

The `executableType` indicates what type of executable is used. VOTCA has different executable types based on the kind of calculation, is the calculaton based on an MD topology, does it need parallelization support etc. We will see the different types shortly.

`whichCalculationToRun` indicates what should be done, a DFT-GWBSE calculation, a file conversion etc. Finally the `optionsFile` is the file with all the input options. 

Lets look at the `log2mps_water.xml` option file

```xml
<options>
	<log2mps>
		<dftpackage>orca</dftpackage>
		<input>DFT_ORCA/water/chelpg.log</input>
		<output>MP_FILES/water_n.mps</output>
	</log2mps>
</options>
```

We see that there are three options to set. The input file (which is the orca output log in this case), the name of the output file and which DFT package was used to do the calculation.

### How to find out which options are available?

There are two ways to find out which options are available for a tool. Every tool has its own description, you can get it for the `log2mps` tool by running the following command (where `-d` means describe).

In [7]:
!xtp_tools -d log2mps

 log2mps:   Generates an mps-file (with polar-site definitions) from a QM log-file
   OPTION      DEFAULT    UNIT           DESCRIPTION
   job_name    (system)                  Input file name without extension, also used for intermediate files
   dftpackage  (orca)                    QM package name
   input       (OPTIONAL)                dftpackage log file to read from, otherwise use job_name
   output      (OPTIONAL)                MPs file with charges, if not given use job_name
Done - stopping here


Alternatively you can generate a starting xml file (with help messages) using `xtp_tools -p <tool_name> -o <output>.xml`, which you can manually edit to fit your needs. This file will contain all available options, mandatory and optional. For `log2mps` it can be generated and viewed with the following commands. Note that the result is much better when the option file is viewed in an editor with syntax highlighting (VS code for example).

In [9]:
!xtp_tools -p log2mps -o temp_log2mpsOptions.xml
!echo "**************"
!cat temp_log2mpsOptions.xml

Writing options for calculator log2mps to temp_log2mpsOptions.xml
Done - stopping here
**************
<options>
<log2mps help="Generates an mps-file (with polar-site definitions) from a QM log-file" label="tool:log2mps">
	<job_name default="system" help="Input file name without extension, also used for intermediate files">system</job_name>
	<dftpackage choices="orca" default="orca" help="QM package name">orca</dftpackage>
	<input default="OPTIONAL" help="dftpackage log file to read from, otherwise use job_name"/>
	<output default="OPTIONAL" help="MPs file with charges, if not given use job_name"/>
</log2mps>
</options>



### The result of `log2mps`

The result of `log2mps` is an mps file, for water it will look like

```text
! GENERATED BY VOTCA::XTP::::LOG2MPS (log-file='../DFT_ORCA/water/chelpg.log' : 3 QM atoms)
! N=3 Q[e]=+0.0000000
Units angstrom
  O +0.0000000 +0.0000000 -0.0043320 Rank 0
    -0.7585550
     P +0.8370000
  H +0.7614610 +0.0000000 +0.5795250 Rank 0
    +0.3787500
     P +0.4960000
  H -0.7614610 +0.0000000 +0.5795250 Rank 0
    +0.3798050
     P +0.4960000
```

We see the atom positions, followed by their partial charge (rank 0, for higher ranks dipoles and quadrupoles are also specified) and the line with the `P` indicating the polarizability. But something is off with the polarizability it is just a single number. 

### 3. Fitting Polarizabilities
To obtain the "real" polarizability we will fit the atomic polarizabilities, such that they represent the total molecular polarizability (computed with ORCA in this case) as close as possible. This is done with the `molpol` tool. Lets run the tool.

In [18]:
!xtp_tools -e molpol -o OPTIONS/molpol_water.xml

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 09:04:09)
votca_csg, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 08:59:16)
votca_tools, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 08:58:17)

Initializing tool
... molpol 
 ... Using package <orca>
 ... Getting polarizabilityEvaluating tool
... molpol  Using 1 threads
Iteration 1 of 100 target:6.46783 current:9.41669

Iteration 2 of 100 target:6.46783 current:6.89261

Iteration 3 of 100 target:6.46783 current:6.54134

Iteration 4 of 100 target:6.46783 current:6.48093

Iteration 5 of 100 target:6.46783 current:6.47018

Iteration 6 of 100 target:6.46783 current:6.46825

... ... Iterative refinement : *CONVERGED*
... ... Scaling coefficient  : -6.50073e-05
First principle polarization [A^3]
 1.18054        0        0
       0 0.772045       -0
       0       -0 0.96597

The options for this tool are the original mps file, the name of the new mps file, which package is used for the polarization calculation and where to find the logfile with the molecular polarizability information. The option file:
```xml
<options>
	<molpol>
		<input>MP_FILES/water_n.mps</input>
		<output>MP_FILES/water_n_pol.mps</output>
		<mode>qmpackage</mode>
		<qmpackage>orca</qmpackage>
		<logfile>DFT_ORCA/water/chelpg.log</logfile>
	</molpol>
</options>
```

If you check the output file now, you will see that it added atomic polarizabilities.

In [24]:
!cat MP_FILES/water_n_pol.mps

! GENERATED BY VOTCA::XTP::MOLPOL (OPTIMIZED)
! N=3 Q[e]=+0.0000000
Units angstrom
  O +0.0000000 +0.0000000 -0.0043320 Rank 0
    -0.7585550
     P +0.5320935 +0.0000000 +0.0000000 +0.5320935 +0.0000000 +0.5320935
  H +0.7614610 +0.0000000 +0.5795250 Rank 0
    +0.3787500
     P +0.3153146 +0.0000000 +0.0000000 +0.3153146 +0.0000000 +0.3153146
  H -0.7614610 +0.0000000 +0.5795250 Rank 0
    +0.3798050
     P +0.3153146 +0.0000000 +0.0000000 +0.3153146 +0.0000000 +0.3153146


### Repeating the process for other molecules

This process needs to be repeated for all molecules in the system. Generating the files for acetone for example can be done by running the following

In [23]:
!xtp_tools -e log2mps -o OPTIONS/log2mps_acetone.xml
!echo " " # we only need it to clearly separate the two commands
!xtp_tools -e molpol -o OPTIONS/molpol_acetone.xml

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 09:04:09)
votca_csg, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 08:59:16)
votca_tools, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 08:58:17)

Initializing tool
... log2mps 
... ... DFT_ORCA/acetone/chelpg.log => MP_FILES/acetone_n.mps
Evaluating tool
... log2mps  Using 1 threads
... ... Using package <orca>
... ... Parsing DFT_ORCA/acetone/chelpg.log
... ... Getting the coordinates
... ... Getting charges
... ... Getting charges
... ... 10 QM atoms, total charge Q = 1e-06 
========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 09:04:09)
votca_csg, version 2022-dev gitid: 24decaf (dirty) (compiled Jul 15 2021, 08:59:16)
votca_tools, versio

## Putting everything together

Now that we have all the input files we need to combine them with the snap shot of the MD trajectory. This is done in the mapping procedure. For this mapping we need to create an input file, the *mapping file*. This file tells VOTCA how to map the multipole data or the groundstate geometry to the MD trajectory. The file itself looks quite complicated so we start by looking only at a small part that will give us the main idea.

Suppose we want to map the ground state geometry of water 

```xml
<mdatoms>2:O76:10 2:H77:11 2:H77:12</mdatoms>
<qmatoms>0:O 1:H 2:H</qmatoms>
<localframe>0 1 2</localframe>

```

In the xml lines above 